In [1]:
# Add related paths to system path
import sys
sys.path.append("../src")

import numpy as np
import pandas
from models import FDBaseline, PDABaseline, CDABaseline
from evaluate import Evaluator

In [9]:
# Load data
train = pandas.read_csv("../data/train.csv", delimiter=',')
val = pandas.read_csv("../data/val.csv", delimiter=',')
test = pandas.read_csv("../data/test.csv", delimiter=',')
complete = pandas.concat([train, val, test])

/Users/mrlsdvd/Projects/Environments/.warfarin_dose_env/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [3]:
# Define baseline models
fixed = FDBaseline(1) # Define fixed dosage to be 1 (5mg / day)
pharm = PDABaseline()
clinic = CDABaseline()

In [11]:
# Get data for all baselines

# Fixed dosage baseline
fixed_train_features = fixed.get_features(train)
fixed_val_features = fixed.get_features(val)
fixed_test_features = fixed.get_features(test)
fixed_complete_features = fixed.get_features(complete)

# Pharmacogenetic dosage baseline
pharm_train_features = pharm.get_features(train)
pharm_val_features = pharm.get_features(val)
pharm_test_features = pharm.get_features(test)
pharm_complete_features = pharm.get_features(complete)

# Clinical dosage baseline
clinic_train_features = clinic.get_features(train)
clinic_val_features = clinic.get_features(val)
clinic_test_features = clinic.get_features(test)
clinic_complete_features = clinic.get_features(complete)

# Get targets for each set (same targets for each baseline)
targets_train = fixed.get_targets(train)
targets_val = fixed.get_targets(val)
targets_test = fixed.get_targets(test)
targets_complete = fixed.get_targets(complete)

In [12]:
# Define evaluators for each model
fixed_eval = Evaluator(fixed)
pharm_eval = Evaluator(pharm)
clinic_eval = Evaluator(clinic)

In [16]:
# Evaluate fixed baseline
fixed_train_acc = fixed_eval.accuracy(fixed_train_features.values, targets_train.values)
print("train acc: " + str(fixed_train_acc))

fixed_val_acc = fixed_eval.accuracy(fixed_val_features.values, targets_val.values)
print("val acc: " + str(fixed_val_acc))

fixed_test_acc = fixed_eval.accuracy(fixed_test_features.values, targets_test.values)
print("test acc: " + str(fixed_test_acc))

fixed_complete_acc = fixed_eval.accuracy(fixed_complete_features.values, targets_complete.values)
print("Total acc: " + str(fixed_complete_acc))

train acc: 0.6123925825418363
val acc: 0.5895117540687161
test acc: 0.6292947558770343
Total acc: 0.611794500723589


In [19]:
# Evaluate Pharmacogenetic baseline
pharm_train_acc = pharm_eval.accuracy(pharm_train_features.values[~np.isnan(pharm_train_features).any(axis=1)], targets_train.values[~np.isnan(pharm_train_features).any(axis=1)])
print("train acc: " + str(pharm_train_acc))

pharm_val_acc = pharm_eval.accuracy(pharm_val_features.values[~np.isnan(pharm_val_features).any(axis=1)], targets_val.values[~np.isnan(pharm_val_features).any(axis=1)])
print("val acc: " + str(pharm_val_acc))

pharm_test_acc = pharm_eval.accuracy(pharm_test_features.values[~np.isnan(pharm_test_features).any(axis=1)], targets_test.values[~np.isnan(pharm_test_features).any(axis=1)])
print("test acc: " + str(pharm_test_acc))

pharm_complete_acc = pharm_eval.accuracy(pharm_complete_features.values[~np.isnan(pharm_complete_features).any(axis=1)], targets_complete.values[~np.isnan(pharm_complete_features).any(axis=1)])
print("Total acc: " + str(pharm_complete_acc))

train acc: 0.6838983050847458
val acc: 0.7044444444444444
test acc: 0.6885245901639344
Total acc: 0.6864387593389178


In [18]:
# Evaluate Clinical baseline
clinic_train_acc = clinic_eval.accuracy(clinic_train_features.values[~np.isnan(clinic_train_features).any(axis=1)], targets_train.values[~np.isnan(clinic_train_features).any(axis=1)])
print("train acc: " + str(clinic_train_acc))

clinic_val_acc = clinic_eval.accuracy(clinic_val_features.values[~np.isnan(clinic_val_features).any(axis=1)], targets_val.values[~np.isnan(clinic_val_features).any(axis=1)])
print("val acc: " + str(clinic_val_acc))

clinic_test_acc = clinic_eval.accuracy(clinic_test_features.values[~np.isnan(clinic_test_features).any(axis=1)], targets_test.values[~np.isnan(clinic_test_features).any(axis=1)])
print("test acc: " + str(clinic_test_acc))

clinic_complete_acc = clinic_eval.accuracy(clinic_complete_features.values[~np.isnan(clinic_complete_features).any(axis=1)], targets_complete.values)
print("Total acc: " + str(clinic_complete_acc))

train acc: 0.6477401129943503
val acc: 0.6622222222222223
test acc: 0.6463700234192038


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets